## BlackJack Game

In [3]:
#Step 1 : Global Variables 

In [1]:
import random

suits = ("Hearts", 'Diamonds', 'Spades', 'Clubs')

ranks = ('Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten',
         'Jack','Queen','King','Ace')

values = {'Two':2,'Three':3,'Four':4,'Five':5,'Six':6,'Seven':7,'Eight':8,'Nine':9,'Ten':10,
          'Jack':10,'Queen':10,'King':10,'Ace':11}

playing = True

In [2]:
# Step 2 : Card

In [4]:
class Card:
    
    def __init__(self,suit,rank):
        
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        return self.rank + ' of ' + self.suit

In [5]:
# Step 3: Deck

In [6]:
class Deck:
    
    def __init__(self):
        self.deck = []
        
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))
                
    def __str__(self):
        deck_comp = ''
        for card in self.deck:
            deck_comp += '\n' + card.__str__() 
        return "The Deck has: " + deck_comp
        
    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        single_card = self.deck.pop()
        return single_card

In [7]:
# Lets see what our Deck looks like!

In [8]:
test_deck = Deck()
print(test_deck)

The Deck has: 
Two of Hearts
Three of Hearts
Four of Hearts
Five of Hearts
Six of Hearts
Seven of Hearts
Eight of Hearts
Nine of Hearts
Ten of Hearts
Jack of Hearts
Queen of Hearts
King of Hearts
Ace of Hearts
Two of Diamonds
Three of Diamonds
Four of Diamonds
Five of Diamonds
Six of Diamonds
Seven of Diamonds
Eight of Diamonds
Nine of Diamonds
Ten of Diamonds
Jack of Diamonds
Queen of Diamonds
King of Diamonds
Ace of Diamonds
Two of Spades
Three of Spades
Four of Spades
Five of Spades
Six of Spades
Seven of Spades
Eight of Spades
Nine of Spades
Ten of Spades
Jack of Spades
Queen of Spades
King of Spades
Ace of Spades
Two of Clubs
Three of Clubs
Four of Clubs
Five of Clubs
Six of Clubs
Seven of Clubs
Eight of Clubs
Nine of Clubs
Ten of Clubs
Jack of Clubs
Queen of Clubs
King of Clubs
Ace of Clubs


In [9]:
# Lets Shuffle

In [12]:
test_deck = Deck()
test_deck.shuffle()
print(test_deck)

The Deck has: 
Three of Diamonds
Ten of Hearts
King of Spades
Three of Hearts
Two of Spades
Jack of Spades
Six of Hearts
Nine of Clubs
Five of Hearts
Five of Diamonds
Four of Clubs
Four of Hearts
Ace of Hearts
Two of Clubs
King of Clubs
Seven of Diamonds
Six of Diamonds
Eight of Clubs
Four of Diamonds
King of Diamonds
Seven of Clubs
Queen of Clubs
Three of Spades
Jack of Clubs
Ten of Clubs
Jack of Hearts
Two of Diamonds
Eight of Diamonds
Seven of Spades
Two of Hearts
Six of Spades
King of Hearts
Five of Clubs
Queen of Hearts
Eight of Hearts
Seven of Hearts
Four of Spades
Five of Spades
Ace of Diamonds
Six of Clubs
Nine of Hearts
Nine of Spades
Eight of Spades
Queen of Diamonds
Three of Clubs
Ten of Diamonds
Ace of Spades
Jack of Diamonds
Queen of Spades
Nine of Diamonds
Ten of Spades
Ace of Clubs


In [13]:
# Step 4: Hand

In [14]:
class Hand:
    
    def __init__(self): 
        self.cards = [] # start with an empty list as we did in Deck Class
        self.value = 0  # start with zero
        self.aces = 0   # add an attribute to keep track of aces
        
    def add_card(self,card):
        # card passed-in from Deck.deal() --> single Card(suit,rank)
        self.cards.append(card)
        self.value += values[card.rank]
        
        # track aces
        if card.rank == "Ace":
            self.aces += 1
        
    def adjust_for_ace(self):
        
        # if total value > 21 and i still have an Ace
        # then change my Ace to be a 1 instead of An 11
        while self.value>21 and self.aces:   # zero(0) is False And other no. is True
            self.value -= 10
            self.aces -= 1

In [15]:
# Some Basics

test_deck = Deck()
test_deck.shuffle()

# Player
test_player = Hand()

# Deal 1 card from the Deck Card(suit,rank)
pulled_card = test_deck.deal()
print(pulled_card)

test_player.add_card(pulled_card)
print(test_player.value)

Queen of Diamonds
10


In [16]:
test_player.add_card(test_deck.deal())

In [17]:
test_player.value

20

In [18]:
# Step 5: Chips

In [19]:
class Chips:
    
    def __init__(self,total=100):
        self.total = total # Set to Default value (it may be supplied by user input) 
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
        
    def loose_bet(self):
        self.total -= self.bet

In [20]:
# Step 6: taking bet

In [21]:
def take_bet(chips):
    
    while True:
        
        try:
            chips.bet = int(input("How many chips would you like to bet?  ")) 
            
        except:
            print("Sorry please provide an integer value!")
            
        else:
            if chips.bet > chips.total:
                print(f"Sorry You dont have enough chips, You have: {chips.total}")
            else:
                break

In [22]:
# Step 7: Hits 

In [23]:
def hit(deck,hand):
    
    single_card = deck.deal()
    hand.add_card(single_card)
    hand.adjust_for_ace()

In [24]:
# Step 8: Hit or Stand

In [25]:
def hit_or_stand(deck,hand):
    
    global playing  # to control upcoming while loop
    
    while True:
        x = input("Hit or Stand? Enter h or s: ")
        
        if x[0].lower()=='h':
            hit(deck,hand)
            
        elif x[0].lower()=='s':
            print("Player stand, dealers turn\n")
            playing = False
        
        else:
            print("Sorry I did not understand that, Please Enter h or s only ")
            continue
            
        break        

In [26]:
# Step 9: Display cards

In [27]:
def show_some(player,dealer):
    print("\nDealer Hand: ")
    print("One Card Hidden! ")
    print(dealer.cards[1])
    print('\n')
    
    print("Player Hand:")
    for card in player.cards:
        print(card)
    
def show_all(player,dealer):
    print("\nDealer Hand: ")
    for card in dealer.cards:
        print(card)
    print('\n')
    
    print("Player Hand:")
    for card in player.cards:
        print(card)

In [28]:
# Step 10: Handle End of Game Scenarios

In [29]:
def player_busts(player,dealer,chips):
    print('Bust Player!')
    chips.loose_bet()
    
def player_wins(player,dealer,chips):
    print('Player Wins!')
    chips.win_bet()
    
def dealer_busts(player,dealer,chips):
    print('Player Wins! Dealer Bust!')
    chips.win_bet()
    
def dealer_wins(player,dealer,chips):
    print('Dealer Wins!')
    chips.loose_bet()
    
def push(player,dealer):
    print('Dealer and Player Tie! Push')

# Now On To The Game!!

## Final Game Play Script

In [30]:
while True:
    
    # Print an Opening Statement
    
    print("\nWelcome to BLACKJACK GAME")
    
    # Create And Shuffle the deck, deal two cards to each players
    deck = Deck()
    deck.shuffle()
    
    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    # Set up the Player's Chips
    player_chips = Chips()
    
    # Prompt the Player for their bet
    take_bet(player_chips)    
    
    # Show Cards (but keep one dealer card hidden)
    show_some(player_hand,dealer_hand)
    
    while playing:       # recall this variable from our hit_or_stand func
        
        # Prompt for player to hit or stand
        hit_or_stand(deck,player_hand)
        
        # Show cards (but keep one dealer card hidden)      
        show_some(player_hand,dealer_hand)
        
        # If player's hand exceeds 21, run player_busts() and break out of loop
        if player_hand.value > 21:
            player_busts(player_hand, dealer_hand, player_chips)
            
            break
        
    # If player has'nt busted, play dealers hand untill dealer reaches 17
    if player_hand.value <= 21:
        
        
        while dealer_hand.value < 17:   #  dealer_hand.value < player_hand.value INSTEAD OF 17
            hit(deck,dealer_hand)
            
        # Show all cards
        show_all(player_hand,dealer_hand)
        
        # Run different winning scenarios
        if dealer_hand.value > 21:
            dealer_busts(player_hand,dealer_hand,player_chips)
            
        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand,dealer_hand,player_chips)
            
        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand,dealer_hand,player_chips)
            
        else:
            push(player_hand,dealer_hand)
            
    
    # Inform Player of their total chips
    print(f"\nPlayer total chips are at: {player_chips.total}")
    
    # Ask to play again  
    new_game = input("Would you like to play another hand? y/n: ").lower()
    
    if new_game[0] == 'y':
        playing = True
        continue
    
    else:
        print("Thank You for Playing!")
        break


Welcome to BLACKJACK GAME
How many chips would you like to bet?  100

Dealer Hand: 
One Card Hidden! 
Queen of Diamonds


Player Hand:
Jack of Diamonds
King of Spades
Hit or Stand? Enter h or s: s
Player stand, dealers turn


Dealer Hand: 
One Card Hidden! 
Queen of Diamonds


Player Hand:
Jack of Diamonds
King of Spades

Dealer Hand: 
Five of Hearts
Queen of Diamonds
Ace of Diamonds
Three of Hearts


Player Hand:
Jack of Diamonds
King of Spades
Player Wins!

Player total chips are at: 200
Would you like to play another hand? y/n: y

Welcome to BLACKJACK GAME
How many chips would you like to bet?  100

Dealer Hand: 
One Card Hidden! 
King of Diamonds


Player Hand:
Queen of Hearts
Seven of Clubs
Hit or Stand? Enter h or s: h

Dealer Hand: 
One Card Hidden! 
King of Diamonds


Player Hand:
Queen of Hearts
Seven of Clubs
Two of Diamonds
Hit or Stand? Enter h or s: s
Player stand, dealers turn


Dealer Hand: 
One Card Hidden! 
King of Diamonds


Player Hand:
Queen of Hearts
Seven of Club